# scrape_caranddriver_wiki
Scrape model, volume, weight and category from car and driver
Get the model and category for suvs from wikipedia

saves into car_category.pkl and car_category_wiki.csv

In [1]:
from urllib.request import urlopen as u_req
import urllib
from bs4 import BeautifulSoup as Soup
import pandas as pd
import requests
from IPython.core.debugger import set_trace
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, InvalidSelectorException
import pickle
import numpy as np
from permile import get_soup

# Define functions

In [2]:
def extract_3_columns(tbl_):
#     Extract columns 1,3 and 4 from a table (wikipedia suv page)
    rows = tbl_.findChildren(['tr'])
    i_row = 0
    column1 = []
    column2 = []
    column4 = []
    for row in rows:
        i_row += 1  
        cells = row.findChildren('td')
        try:
            column1.append(cells[0].string)
            column2.append(cells[1].string)            
            column4.append(cells[3].string)            
        except:
            print('no string in row {}'.format(i_row))
    return column1, column2, column4


In [3]:
def car_and_driver_data(url_):
#     Get car names (make and model), passenger volume, and curb weight for all cars in 
#     the caranddriver.com page passed
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_)
    scroll_to_bottom(driver)
    hdrs2 = driver.find_elements_by_tag_name("h2")
    names = [x.text for x in hdrs2]
    
    anchors = driver.find_elements_by_tag_name("a")
    class_name = 'pointer mr3 f14 sserif-2 text-denim tr-text-2 underline'
    mask = np.asarray([(x.get_attribute('class')==class_name) for x in anchors])
    links = [x.get_attribute('href') for x in np.asarray(anchors)[mask]]
    volume = []
    weight = []
    epa_classification = []
    count = 0
    for i_vehicle in links:
        print('Scraping {}...'.format(names[count]))
        count += 1
        driver.get('{}/specs'.format(i_vehicle))
        table_data = driver.find_elements_by_tag_name("td")
        volume.append(get_table_value(table_data,'Passenger Volume : ft³'))
        weight.append(get_table_value(table_data,'Base Curb Weight : lbs'))
        epa_classification.append(get_table_value(table_data,'EPA Classification :'))
    
    return pd.DataFrame(dict(zip(['model','volume','weight','epa_classification'],[names,volume,weight,epa_classification])))

In [4]:
def scroll_to_bottom(driver_):
#     Scroll to the bottom of the caranddriver.com page, click "show more cars" and repeat
#     until there are no more cars to be shown
    down_scroll = True
    while down_scroll:    
        driver_.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:         
            btn = driver_.find_element_by_xpath('//*[@id="catContentContainer"]/div[2]/div/div/div[1]/button')    
            btn.click()
        except (NoSuchElementException,InvalidSelectorException):
            down_scroll = False
        except:
            print('Uncaught exception, figure it out!')
            down_scroll = False

In [5]:
def get_table_value(table_data,parameter):
#     Get the 2nd column table value, given a table and the text content of the first column
    try:
        title_element = np.asarray(table_data)[np.asarray([(x.text == parameter) for x in table_data])][0]
    except IndexError:
        return np.nan
    parameter_text = title_element.find_elements_by_xpath('../td')[1].text
    try:        
        parameter_value = float(parameter_text)
    except ValueError:
        return parameter_text
    return parameter_value

# Get SUVs from wikipedia list

In [6]:
# suv_url = 'https://en.wikipedia.org/wiki/List_of_sport_utility_vehicles'
# soup = get_soup(suv_url)
# tbl = soup.find_all('table')

In [7]:
# target_tbl = tbl[0]
# makes, models, class_name = extract_2_columns(target_tbl)
# wiki_suv = pd.DataFrame({'make':makes, 'models':models, 'class': class_name})


# Get Car and Driver Data

In [8]:
# URLs for the various car categories
urls = ['https://www.caranddriver.com/best-sedans',\
       'https://www.caranddriver.com/best-small-cars',\
       'https://www.caranddriver.com/best-suvs-crossovers',\
       'https://www.caranddriver.com/best-pickup-trucks',\
       'https://www.caranddriver.com/best-minivans-vans',\
       'https://www.caranddriver.com/best-station-wagons',\
       'https://www.caranddriver.com/best-luxury-cars']
# Car categories (for naming dataframes)
categories = ['sedans','small_cars','crossovers','trucks',\
              'vans','station_wagons','luxury_cars']

In [9]:
#Scrape (takes a couple hours or so)
df = []
for i_cat in range(len(urls)):
    print('Scraping all {}\n\n'.format(categories[len(df)]))
    df.append(car_and_driver_data(urls[i_cat]))    

Scraping all sedans


Scraping Mercedes-Benz S-class...
Scraping Alfa Romeo Giulia Quadrifoglio...
Scraping BMW M5...
Scraping Cadillac CTS-V...
Scraping Honda Accord...
Scraping Honda Civic...
Scraping Porsche Panamera Turbo / Turbo S...
Scraping Audi RS3...
Scraping Mercedes-AMG C43...
Scraping Mercedes-AMG C63...
Scraping Mercedes-AMG E43...
Scraping Mercedes-AMG E63 S 4Matic...
Scraping Mercedes-AMG S63 / S65...
Scraping Toyota Yaris iA...
Scraping Alfa Romeo Giulia...
Scraping Honda Civic Si...
Scraping Hyundai Accent...
Scraping Audi S3...
Scraping Audi A4...
Scraping Mercedes-Benz E-class...
Scraping Mazda Mazda 3...
Scraping Mazda Mazda 6...
Scraping Porsche Panamera...
Scraping Audi A5 Sportback...
Scraping Chevrolet Malibu...
Scraping Nissan Maxima...
Scraping Audi A6...
Scraping Mercedes-Benz C-class...
Scraping Kia Stinger...
Scraping Kia Rio...
Scraping Genesis G90...
Scraping Cadillac CTS...
Scraping Audi A3...
Scraping Tesla Model S...
Scraping Kia Cadenza...
Scraping Au

Scraping BMW X4...
Scraping Nissan Murano...
Scraping Maserati Levante...
Scraping Cadillac Escalade / Escalade ESV...
Scraping Buick Encore...
Scraping Buick Enclave...
Scraping Nissan Juke NISMO / NISMO RS...
Scraping Mercedes-Benz GLC Coupe...
Scraping Mercedes-Benz GLE-class...
Scraping Mini Cooper Countryman JCW...
Scraping Nissan Armada...
Scraping Mini Cooper Countryman / S...
Scraping Audi Q3...
Scraping Land Rover Discovery Sport...
Scraping Land Rover Range Rover Evoque...
Scraping Infiniti QX30...
Scraping Infiniti QX50...
Scraping Infiniti QX60...
Scraping Subaru Forester...
Scraping Infiniti QX70...
Scraping Infiniti QX80...
Scraping Nissan Rogue...
Scraping Lincoln MKC...
Scraping Ford Flex...
Scraping Lincoln MKX...
Scraping Nissan Pathfinder...
Scraping Hyundai Santa Fe...
Scraping Ford EcoSport...
Scraping Ford Explorer...
Scraping Buick Envision...
Scraping Toyota Highlander...
Scraping Fiat 500X...
Scraping Mercedes-AMG GLE43 Coupe 4Matic / GLE63 S Coupe 4Matic...
Sc

Scraping Buick Envision...
Scraping Mercedes-AMG CLA45 4Matic...
Scraping Mini Cooper Countryman / S...
Scraping Lexus GS...
Scraping BMW 6-series Gran Turismo...
Scraping Lexus GX...
Scraping Audi Q3...
Scraping Mercedes-AMG GLE43 Coupe 4Matic / GLE63 S Coupe 4Matic...
Scraping Acura RLX...
Scraping Mercedes-Benz G-class...
Scraping Mercedes-Benz CLS-class...
Scraping Acura ILX...
Scraping Infiniti Q70...
Scraping Toyota Land Cruiser...
Scraping Lexus LX...
Scraping Lexus ES...
Scraping Lincoln MKT...
Scraping Tesla Roadster...
Scraping Mercedes-AMG GT63 / GT63 S...
Scraping Land Rover Discovery SVX...
Scraping Genesis G70...
Scraping Jaguar I-Pace...
Scraping Bentley Continental Supersports...
Scraping Mercedes-Benz A-class...
Scraping BMW 8-series...
Scraping Lexus UX...
Scraping BMW M8 Gran Coupe...
Scraping Lincoln Nautilus...
Scraping Lamborghini Urus...
Scraping Polestar Polestar 1...
Scraping Jaguar F-Pace SVR...
Scraping Lincoln Aviator...
Scraping Volkswagen Arteon...
Scrapin

In [10]:
# Concatenate the resulting dataframes
car_and_driver_all = pd.concat(df,keys=categories)

# Export

In [11]:
# with open('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/car_category2.pkl','wb') as f:
#     pickle.dump([sedans,small_cars,crossovers, trucks, vans, station_wagons, luxury_cars],f)
# # wiki_suv.to_csv('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/car_category_wiki.pkl')    

In [12]:
with open('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/car_category2.pkl','wb') as f:
    car_and_driver_all.to_pickle(f)

In [ ]:
with open('/Users/ahakso/odrive/agl/perMileCost/car_category.pkl','rb') as f:
    df = pickle.load(f)

In [ ]:
df = pd.read_csv('/Users/ahakso/Documents/gitDir/permileFlask/mysite/static/car_category_wiki.pkl')